In [44]:
import gc
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import nlpaug.augmenter.word as naw
import preprocessor as p
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification, MarianTokenizer, MarianMTModel
from tqdm import tqdm
from sklearn import metrics

In [45]:
train_df = pd.read_csv("./data/train.csv")
test_df = pd.read_csv("./data/test.csv")

In [46]:
train_df = train_df[['keyword', 'text', 'target']]
test_df = test_df[['keyword', 'text']]

train_df['keyword'] = train_df['keyword'].fillna('None')
test_df['keyword'] = test_df['keyword'].fillna('None')

In [57]:
def extend_clean(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'\b(?:gt)+\b', '', text)
    return text

p.set_options(p.OPT.URL, p.OPT.MENTION, p.OPT.EMOJI, p.OPT.HASHTAG, p.OPT.NUMBER, p.OPT.SMILEY)
train_df['text'] = train_df['text'].apply(lambda x: extend_clean(p.clean(x).lower()))
test_df['text'] = test_df['text'].apply(lambda x: extend_clean(p.clean(x).lower()))

In [ ]:
# aug = naw.ContextualWordEmbsAug(
#     model_path='xlm-roberta-base',
#     action='substitute'
# )
# aug_rows = []

# for idx, row in train_df.iterrows():
#     aug_text = aug.augment(row['text'])
#     new_row = {
#         'keyword': row['keyword'],
#         'text': aug_text,
#         'target': row['target']
#     }
#     aug_rows.append(new_row)

# aug_df = pd.DataFrame(aug_rows)
# new_df = pd.concat([train_df, aug_df], ignore_index=True)
# new_df.to_csv('./data/train.csv', index=False)

In [ ]:
# def translate(texts, src_lang, out_lang, batch_size=32):
#     model_name = f'Helsinki-NLP/opus-mt-{src_lang}-{out_lang}'
#     model = MarianMTModel.from_pretrained(model_name).to('cuda')
#     tokenizer = MarianTokenizer.from_pretrained(model_name)
     
#     translated_texts = []
#     for i in tqdm(range(0, len(texts), batch_size)):
#         batch = texts[i:i+batch_size]
#         inputs = tokenizer(batch, return_tensors='pt', padding=True, truncation=True).to('cuda')
#         with torch.no_grad():
#             translated = model.generate(**inputs, early_stopping=True, max_length=128)
#         outputs = tokenizer.batch_decode(translated, skip_special_tokens=True)
#         translated_texts.extend(outputs)
        
#     return translated_texts

# translated_df = train_df.copy()
# translated_df['text'] = translate(translated_df['text'].tolist(), 'en', 'de', batch_size=32)
# translated_df['text'] = translate(translated_df['text'].tolist(), 'de', 'en', batch_size=32)
# new_df2 = pd.concat([train_df, translated_df], ignore_index=True)
# new_df2.to_csv('./data/train.csv', index=False)

100%|██████████| 476/476 [15:20<00:00,  1.93s/it]


In [58]:
class CustomDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=128, is_test=False):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.is_test = is_test
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        keyword = str(self.df.iloc[index]['keyword'])
        text = str(self.df.iloc[index]['text'])
        
        comb_text = keyword + " : " + text
        
        encoding = self.tokenizer(
            comb_text,
            padding='max_length',
            truncation=True,
            max_length=self.max_len,
            return_tensors='pt'
        )
        
        inputs = {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0)
        }
        
        if not self.is_test and 'target' in self.df.columns:
            labels = torch.tensor(self.df.iloc[index]['target'], dtype=torch.long)
            return inputs, labels
        else:
            return inputs

In [59]:
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')
train_val_dataset = CustomDataset(train_df, tokenizer)
test_dataset = CustomDataset(test_df, tokenizer, is_test=True)

train_length = int(len(train_val_dataset) * 0.8)
val_length = len(train_val_dataset) - train_length

train_dataset, val_dataset = random_split(train_val_dataset, [train_length, val_length])

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [50]:
def train_model(model,
        train_dataloader,
        val_dataloader,
        optimizer, 
        scheduler,
        num_epochs=10,
        criterion=None,
        device=None
    ):
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    if criterion is None:
        criterion = torch.nn.CrossEntropyLoss()
    
    model = model.to(device)
    torch.backends.cudnn.benchmark = True
    scaler = torch.GradScaler(device)

    for epoch in range(num_epochs):
        model.train()

        running_loss = 0.0
        all_labels = []
        all_preds = []
        
        tqdm_train = tqdm(train_dataloader, desc=f"Training epoch {epoch + 1}: ", leave=False)
        
        for inputs, labels in tqdm_train:
            if isinstance(inputs, dict):
                inputs = {k: v.to(device) for k, v in inputs.items()}
            else:
                inputs = inputs.to(device)
            
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            with torch.autocast(device_type="cuda"):
                outputs = model(**inputs)
                loss = criterion(outputs.logits, labels)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            preds = torch.argmax(outputs.logits, dim=1)
            running_loss += loss.item()
            
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
            
            tqdm_train.set_postfix(loss=loss.item())
        
        train_loss = running_loss / len(train_dataloader)
        train_f1 = metrics.f1_score(all_labels, all_preds, average="weighted")
        
        model.eval()
        
        running_loss = 0.0
        all_preds = []
        all_labels = []
        
        tqdm_val = tqdm(val_dataloader, desc=f"Validation epoch {epoch + 1}: ", leave=False)
        
        with torch.no_grad():
            for inputs, labels in tqdm_val:
                if isinstance(inputs, dict):
                    inputs = {k: v.to(device) for k, v in inputs.items()}
                else:
                    inputs = inputs.to(device)

                labels = labels.to(device)
                
                with torch.autocast(device_type="cuda"):
                    outputs = model(**inputs)
                    loss = criterion(outputs.logits, labels)
                
                preds = torch.argmax(outputs.logits, dim=1)
                running_loss += loss.item()
                
                all_labels.extend(labels.cpu().numpy())
                all_preds.extend(preds.cpu().numpy())
                
                tqdm_val.set_postfix(loss=loss.item())
        
        val_loss = running_loss / len(val_dataloader)
        val_f1 = metrics.f1_score(all_labels, all_preds, average="weighted")
        
        if isinstance(scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
            scheduler.step(val_loss)
        else:
            scheduler.step()
        
        print(f"--- Epoch {epoch + 1}/{num_epochs}\n"
              f"Train F1: {train_f1:.4f}, Val F1: {val_f1:.4f}\n"
              f"Train loss: {train_loss:.4f}, Val loss: {val_loss:.4f}")

In [69]:
model = RobertaForSequenceClassification.from_pretrained("roberta-large", num_labels=2)

for param in model.parameters():
    param.requires_grad = False

for param in model.classifier.parameters():
    param.requires_grad = True

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=6)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [70]:
train_model(model, train_dataloader, val_dataloader, optimizer, scheduler, num_epochs=6)

--- Epoch 1/6
Train F1: 0.7203, Val F1: 0.7923
Train loss: 0.5721, Val loss: 0.4886


--- Epoch 2/6
Train F1: 0.7841, Val F1: 0.8079
Train loss: 0.4971, Val loss: 0.4466


--- Epoch 3/6
Train F1: 0.7904, Val F1: 0.8108
Train loss: 0.4801, Val loss: 0.4342


--- Epoch 4/6
Train F1: 0.7946, Val F1: 0.8111
Train loss: 0.4660, Val loss: 0.4279


--- Epoch 5/6
Train F1: 0.7961, Val F1: 0.8118
Train loss: 0.4637, Val loss: 0.4262


--- Epoch 6/6
Train F1: 0.7964, Val F1: 0.8113
Train loss: 0.4634, Val loss: 0.4258


In [71]:
def test_model(model, test_dataloader, device=None):
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    model.to(device)
    model.eval()
    
    all_preds = []
    
    with torch.no_grad():
        for inputs in test_dataloader:
            inputs = {k: v.to(device) for k, v in inputs.items()}
            outputs = model(**inputs)
            preds = torch.argmax(outputs.logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
    
    return all_preds

In [72]:
predictions = test_model(model, test_dataloader)

In [73]:
del model
del optimizer
del scheduler

torch.cuda.empty_cache()

gc.collect()

9

In [74]:
test_csv = pd.read_csv('./sample_submission.csv')
test_csv['target'] = predictions
test_csv.to_csv('sample_submission.csv', index=False)